In [1]:
import pandas as pd
import numpy as np

In [2]:
#R library(randomForest)
from sklearn.ensemble import RandomForestRegressor

In [3]:
#R df = read.csv(file="./data/proposed-assessments-as-of-april-18th.csv", header=TRUE, sep=",")
df = pd.read_csv("data/proposed-assessments-as-of-april-18th.csv",low_memory=False)

In [4]:
# orig = read.csv(file="./data/proposed-assessments-as-of-april-18th.csv", header=TRUE, sep=",")
# OMITTED IN PORT

In [5]:
#R df = df[sapply(df, is.numeric)]
# this should select only the numeric columns
df = df.select_dtypes(exclude=['object'])

In [6]:
# insert underscores in column names
df.columns = [c.replace(' ', '_') for c in df.columns]

df.columns = df.columns.str.replace('\n','')
df.head(5)

,BlockNo,SalePrice,2017_AssmtTaxable,2017_AssmtExempt,2017_AssmtTotal,2017_TaxesTotal,YearBuilt,SquareFootage,2018_AssmtTaxable,2018_AssmtExempt,2018_AssmtTotal,2018_TaxesEstimated
0,301.0,390000.0,126000,NaN,126000,9828.0,1925.0,1600.0,416500,NaN,416500,6747.30
1,301.0,1.0,114000,NaN,114000,8892.0,1929.0,1616.0,363300,NaN,363300,5885.46
2,301.0,150000.0,124700,NaN,124700,9726.6,1929.0,1600.0,364900,NaN,364900,5911.38
3,301.0,305000.0,88000,NaN,88000,6864.0,1929.0,1582.0,335000,NaN,335000,5427.00
4,301.0,1.0,122100,NaN,122100,9523.8,1929.0,1582.0,396200,NaN,396200,6418.44


In [7]:
list(df)

['BlockNo',
 'SalePrice',
 '2017_AssmtTaxable',
 '2017_AssmtExempt',
 '2017_AssmtTotal',
 '2017_TaxesTotal',
 'YearBuilt',
 'SquareFootage',
 '2018_AssmtTaxable',
 '2018_AssmtExempt',
 '2018_AssmtTotal',
 '2018_TaxesEstimated']

In [9]:
#R undesired <- c('X2018.Assmt.Taxable', 'X2018.Assmt.Exempt', 'X2018.Assmt.Total')
undesired = ['2017_AssmtExempt', '2018_AssmtTaxable', '2018_AssmtExempt', '2018_AssmtTotal','2018_TaxesEstimated']

# AT undesired = ['BlockNo','2017_AssmtExempt', '2018_AssmtTaxable', '2018_AssmtExempt', '2018_AssmtTotal','2018_TaxesEstimated']


In [10]:
#R idx = !(colnames(df) %in% undesired)
idx = [x for x in list(df) if x not in undesired]
idx

['BlockNo',
 'SalePrice',
 '2017_AssmtTaxable',
 '2017_AssmtTotal',
 '2017_TaxesTotal',
 'YearBuilt',
 'SquareFootage']

In [16]:
#R df = df[,idx]
# creates an index on all the columns except those in undesired
df = df.set_index(idx)

In [17]:
# replace NaN's with median of the column

#R for(i in 1:ncol(df)){
#R   df[is.na(df[,i]), i] = median(df[,i], na.rm = TRUE)
#R }

df = df.fillna(df.mean())

In [18]:
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(df[iris.feature_names], iris.target, test_size=0.5, stratify=iris.target, random_state=123456)
X_train, X_test, y_train, y_test = train_test_split(df[idx],df['2018_TaxesEstimated'], test_size=0.2)

KeyError: "['BlockNo' 'SalePrice' '2017_AssmtTaxable' '2017_AssmtTotal'\n '2017_TaxesTotal' 'YearBuilt' 'SquareFootage'] not in index"

In [15]:
# check for NaN's
# https://datascience.stackexchange.com/questions/11928/valueerror-input-contains-nan-infinity-or-a-value-too-large-for-dtypefloat32

np.where(np.isnan(X_train))


(array([], dtype=int64), array([], dtype=int64))

In [ ]:
#R rf = randomForest(X2018.Taxes.Estimated ~ ., data=df)
rf = RandomForestRegressor(n_estimators=5, random_state=2)
rf.fit(X_train, y_train)

rf.score(X_train,y_train)


In [ ]:
#R diff = predict(rf, df) - df$X2018.Taxes.Estimated
#R x = seq(0,5,0.1)
#R plot(seq(0,5,0.1), sapply(x, function(w) dim(df[abs(diff)>w*sd(diff),])[1]))